# Preprocessing Only Workflow

In this workflow, CFRL takes in an offline trajectory and then preprocesses the offline trajectory 
using `SyntheticPreprocessor`. The final output of the workflow is the preprocessed (debiased) 
offline trajectory. This workflow is appropriate when the user does not want to train policies using 
CFRL. Instead, the user can take the preprocessed trajectory to train a counterfactually fair policy 
using another reinforcement learning library or application that better fits their needs.

We begin by importing the libraries needed for this demonstration.

In [1]:
# Need this temporarily to import CFRL before it is officially published to PyPI
import sys
sys.path.append("E:/learning/university/MiSIL/CFRL Python Package/CFRL")

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from cfrl.reader import read_trajectory_from_dataframe, convert_trajectory_to_dataframe
from cfrl.preprocessor import SequentialPreprocessor
np.random.seed(10) # ensure reproducibility
torch.manual_seed(10) # ensure reproducibility

## Data Loading

In this demonstration, we use an offline trajectory generated from a `SyntheticEnvironment` using some pre-specified transition rules. Although it is actually synthesized, we treat it as if it is from some unknown environment for pedagogical convenience in this demonstration.

The trajectory contains 500 individuals (i.e. $N=500$) and 10 transitions (i.e. $T=10$). The actions are binary ($0$ or $1$) and were sampled using a random policy that selects $0$ or $1$ randomly with equal probability. It is stored in a tabular format in a `.csv` file. The sensitive attribute variable is univariate, stored in the column `z1`. The legit values of the sensitive attribute are $0$ and $1$. The state variable is also univariate, stored in the column `state1`. The actions are stored in the column `action` and rewards in the column `reward`. The tabular data also includes an extra irrelevant column `timestamp`. 

We can load and view the tabular data.

In [3]:
trajectory = pd.read_csv('../data/sample_data_large_uni.csv')
trajectory

,Unnamed: 0,ID,timestamp,z1,action,reward,state1
0,0,1.0,1.0,0.0,NaN,NaN,1.324345
1,1,1.0,2.0,0.0,1.0,1.524345,-0.813722
2,2,1.0,3.0,0.0,1.0,-0.613722,-0.526683
3,3,1.0,4.0,0.0,1.0,-0.326683,-0.464447
4,4,1.0,5.0,0.0,1.0,-0.264447,-2.075518
...,...,...,...,...,...,...,...
5495,5495,500.0,7.0,1.0,1.0,-2.468460,-0.941954
5496,5496,500.0,8.0,1.0,1.0,-1.430345,-2.536595
5497,5497,500.0,9.0,1.0,0.0,-1.068298,-0.946557
5498,5498,500.0,10.0,1.0,0.0,-0.273278,-0.709017


We now read the trajectory from the tabular format into Trajectory Arrays.

In [4]:
zs, states, actions, rewards, ids = read_trajectory_from_dataframe(
                                                data=trajectory, 
                                                z_labels=['z1'], 
                                                state_labels=['state1'], 
                                                action_label='action', 
                                                reward_label='reward', 
                                                id_label='ID', 
                                                T=10
                                                )

## Preprocessor Training

Before preprocessing the trajectory, we need to first train a preprocessor. To mitigate overfitting, we use a random subset of 250 individuals in the trajectory to train the preprocessor. The remaining 250 individuals will be actually preprocessed. We now form these two sets.

In [5]:
(
    zs_train, zs_prepro, 
    states_train, states_prepro, 
    actions_train, actions_prepro, 
    rewards_train, rewards_prepro, 
    ids_train, ids_prepro
) = train_test_split(zs, states, actions, rewards, ids, test_size=0.5)

We now use the training set to train a `SequentialPreprocessor`.

In [6]:
sp = SequentialPreprocessor(z_space=[[0], [1]], 
                            num_actions=2, 
                            cross_folds=1, 
                            mode='single', 
                            reg_model='nn')
sp.train_preprocessor(zs=zs_train, xs=states_train, actions=actions_train, rewards=rewards_train)

100%|██████████| 1000/1000 [00:36<00:00, 27.34it/s]


(array([[[-0.32461696,  0.57096795],
         [ 0.22408737,  1.31944726],
         [-0.75577441,  1.2936311 ],
         ...,
         [-0.65848739,  0.02353077],
         [-0.64196535,  0.73095815],
         [-0.40729995,  1.4571281 ]],
 
        [[-0.2082495 ,  0.6873354 ],
         [ 1.25568126,  2.39690117],
         [-0.45044321,  1.84835638],
         ...,
         [-1.11037296,  0.43440456],
         [-1.58189396,  0.45079704],
         [-2.8786627 , -1.66740244]],
 
        [[ 1.20425904,  2.09984394],
         [-0.4995148 ,  1.66238533],
         [ 1.24682442,  3.91839007],
         ...,
         [ 0.41160054,  2.41311652],
         [-1.37682367,  0.37004053],
         [ 0.36016143,  2.51716889]],
 
        ...,
 
        [[ 1.26530747,  2.16089238],
         [-1.92501584,  0.22994996],
         [-1.00322249,  1.14478895],
         ...,
         [-2.70255556, -1.07172801],
         [-0.83831198,  0.45569243],
         [-0.87000105,  0.26598686]],
 
        [[ 0.93616403,  1.831

## Data Preprocessing

We now preprocess the remaining data that are not in the training set.

In [7]:
states_tilde, rewards_tilde = sp.preprocess_multiple_steps(zs=zs_prepro, 
                                                           xs=states_prepro, 
                                                           actions=actions_prepro, 
                                                           rewards=rewards_prepro)

## Data Exporting

Finally, we convert the preprocessed trajectory back into the tabular format so that it is easier to store and manage. Note that the state variable now becomes bivariate because it includes both the counterfactual state under $Z=0$ and the counterfactual state under $Z=1$. For simplicity, we call the new concatenated counterfactual states as `state1` and `state2`, which is the default option provided by `convert_trajectory_to_dataframe()` (so we do not need to specify the `state_labels` argument).

In [8]:
preprocessed_trajectory = convert_trajectory_to_dataframe(
                                        zs=zs_prepro, 
                                        states=states_tilde, 
                                        actions=actions_prepro, 
                                        rewards=rewards_tilde, 
                                        ids=ids_prepro, 
                                        z_labels=['z1'], 
                                        action_label='action', 
                                        reward_label='reward', 
                                        id_label='ID', 
                                        T_label='time_step'
                                        )
preprocessed_trajectory

,ID,time_step,z1,action,reward,state1,state2
0,152.0,1.0,1.0,NaN,NaN,-2.630423,-1.734838
1,152.0,2.0,1.0,0.0,-0.880539,-0.089101,0.435083
2,152.0,3.0,1.0,0.0,0.021090,-0.067945,0.846006
3,152.0,4.0,1.0,1.0,0.395191,0.660311,2.372561
4,152.0,5.0,1.0,0.0,0.586032,0.450962,2.133603
...,...,...,...,...,...,...,...
2745,278.0,7.0,0.0,0.0,-0.303801,-2.123443,-1.460281
2746,278.0,8.0,0.0,0.0,-0.754010,-1.777098,-1.055384
2747,278.0,9.0,0.0,0.0,-0.579747,-1.132943,-0.185267
2748,278.0,10.0,0.0,1.0,-0.716901,-3.606803,-2.026175


## Alternative: Preprocessing All Individuals

Sometimes, the number of individuals in the trajectory is small. In this case, if we only preprocess a subset of individuals, the resulting preprocessed trajectory might be too small to be useful for policy learning. In this case, we can directly preprocess all individuals using the `train_preprocessor()` function when we set `cross_folds` to a relatively large number.

When `cross_folds=K` where `K` is greater than 1, `train_preprocessor()` will internally divide the training data into `K` folds. For each $i=1,\dots,K$, it trains a transition dynamics model based on all the folds other than the $i$-th one, and this model is then used to preprocess data in the $i$-th fold. This results in `K` folds of preprocessed data, each of which is processed using a model that is trained on the other folds. These `K` folds of preprocessed data are then combined and returned by `train_preprocessor()`. This method allows us to preprocess all individuals in the trajectory while reducing overfitting.

To use this functionality, we first initialize a `SequentialPreprocessor` with `cross_folds` greater than 1. We use `cross_folds=5` here.

In [9]:
sp_cf5 = SequentialPreprocessor(z_space=[[0], [1]], 
                                num_actions=2, 
                                cross_folds=5, 
                                mode='single', 
                                reg_model='nn')

We now simultaneously train the preprocessor and preprocess all individuals in the trajectory using the precedure described above.

In [10]:
states_tilde_cf5, rewards_tilde_cf5 = sp_cf5.train_preprocessor(zs=zs, 
                                                                xs=states, 
                                                                actions=actions, 
                                                                rewards=rewards)

100%|██████████| 1000/1000 [01:25<00:00, 11.66it/s]


We can now convert the preprocessed trajectory into the tabular format.

In [11]:
preprocessed_trajectory_cf5 = convert_trajectory_to_dataframe(
                                            zs=zs, 
                                            states=states_tilde_cf5, 
                                            actions=actions, 
                                            rewards=rewards_tilde_cf5, 
                                            ids=ids, 
                                            z_labels=['z1'], 
                                            action_label='action', 
                                            reward_label='reward', 
                                            id_label='ID', 
                                            T_label='time_step'
                                            )
preprocessed_trajectory_cf5

,ID,time_step,z1,action,reward,state1,state2
0,1.0,1.0,0.0,NaN,NaN,1.324345,2.287555
1,1.0,2.0,0.0,1.0,1.986266,-0.813722,1.552295
2,1.0,3.0,0.0,1.0,0.485075,-0.526683,2.264417
3,1.0,4.0,0.0,1.0,1.061471,-0.464447,2.424813
4,1.0,5.0,0.0,1.0,1.196269,-2.075518,0.895485
...,...,...,...,...,...,...,...
5495,500.0,7.0,1.0,1.0,-2.643738,-2.104338,-0.941954
5496,500.0,8.0,1.0,1.0,-1.650895,-3.882026,-2.536595
5497,500.0,9.0,1.0,0.0,-1.246537,-1.023986,-0.946557
5498,500.0,10.0,1.0,0.0,-0.409462,-1.311622,-0.709017
